In [ ]:
#!apt-get install git
#!pip install opencv-python

## Original Notebook classifying all 4 classes:
- 0 : Glioma Tumor
- 1 : Meningioma Tumor
- 2 : No Tumor
- 3 : Pituitary Tumor

In [ ]:
import sys
sys.path.append('../src/')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import os
import shutil
import imghdr
import random
import mlflow
import Data_Augmentation
import Data_Loader
import Plotting_functions
import Model_Builder
import Data_Cleaning
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.utils import to_categorical

from Data_Loader import DataLoader
from Data_Augmentation import Data_Augmentation
from Data_Cleaning import Cleaner
from Plotting_functions import Plotting_functions
from Model_Builder import CNNModel


In [ ]:
#Initialise class
data_loader = DataLoader()
Augmentator = Data_Augmentation()
ModelBuilder = CNNModel()
Cleaning = Cleaner(ModelBuilder)



### Data Source: [Brain Tumor Classification (MRI)](https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri)

In [ ]:
training_path = '/Users/harjitgakhal/Documents/Github_Repositories/Brain-Tumour-Classification/Images_Brain/Training'
testing_path = '/Users/harjitgakhal/Documents/Github_Repositories/Brain-Tumour-Classification/Images_Brain/Testing'

In [ ]:
# !nvidia-smi

In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

We can see from the above output that Tensorflow is utilising the NVIDIA Tesla T4 GPU.

In [ ]:
# # Avoid OOM errors by setting GPU Memory Consumption Growth
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Checking to see the different folders within our path
for image_class in os.listdir(training_path):
  print(image_class)


In [ ]:
img = cv2.imread(os.path.join(training_path, 'pituitary_tumor', 'p (503).jpg'))
img.shape # We can see the image is a 512x512 with a depth of 3

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB ))
plt.show()

# Loading data

The datset was already split into testing and training data so we do not need to define these later

In [ ]:
# Load the training dataset
Training_data = data_loader.Load_Training_data(training_path)
# Load the validation dataset
Validation_data = data_loader.Load_Validaion_data(training_path)
# Load the testing dataset
Testing_data = data_loader.Load_Testing_data(testing_path)

In [ ]:
# Required to plot the graphs

Training_iterator = Training_data.as_numpy_iterator()
Testing_iterator = Testing_data.as_numpy_iterator()
batch_training = Training_iterator.next()

# Inspect class names
class_names = Training_data.class_names
print("Class Names:", class_names)

Plotter = Plotting_functions(class_names)


In [ ]:
print(batch_training[0].shape) # Images are now of dimension 256 x 256
print(batch_training[1]) # Labels are 0,1,2 and 3 for the different classes

In [ ]:
def plot_tumour(batch_training, label = None):
    fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
    
    for idx, img_index in enumerate([1,13,4,5]):
        ax[idx].imshow(batch_training[0][img_index].astype(int))

        #Take default label or assign to custom names
        if label is None:
            current_label = batch_training[1][img_index]
        else:
            current_label = label[img_index]

        ax[idx].title.set_text(current_label)

    plt.tight_layout()  # Adjust layout to fit everything nicely
    plt.show()

In [ ]:
plot_tumour(batch_training)

#### Labels
-   0 : Glioma Tumor
-   1 : Meningioma Tumor
-   2 : No Tumor
-   3 : Pituitary Tumor


In [ ]:
# fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
# for idx, img_index in enumerate([1,13,4,5]):
#     ax[idx].imshow(batch_training[0][img_index].astype(int))
#     ax[idx].title.set_text(class_names[batch_training[1][img_index]])
#     print(img_index)
#     print(batch_training[1][img_index])

labels = [class_names[label] for label in batch_training[1]]

plot_tumour(batch_training, labels)



# Preprocessing Data

In [ ]:
num_classes = 4

# Load the training dataset
Training_data = Training_data.map(Augmentator.augment_image)
Training_data = Training_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))

# Load the validation dataset
Validation_data = Validation_data.map(Augmentator.augment_image)
Validation_data = Validation_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))

# Load the testing dataset
#Validation_data = Validation_data.map(augment_image)
Testing_data = Testing_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))

# X is Images
# Y is Labels


In [ ]:
batch_training = Training_data.as_numpy_iterator().next()

fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img_index in enumerate([12,13,14,15]):
    ax[idx].imshow(batch_training[0][img_index])
    ax[idx].title.set_text(batch_training[1][img_index])


    batch_training = Training_data.as_numpy_iterator().next()

plot_tumour(batch_training)



#### Labels
-   [1,0,0,0] : Glioma Tumor
-   [0,1,0,0]  : Meningioma Tumor
-   [0,0,1,0]  : No Tumor
-   [0,0,0,1]  : Pituitary Tumor


# Deep Model

In [ ]:
logdir = "/Users/harjitgakhal/Documents/Github_Repositories/Brain-Tumour-Classification/Notebooks/logs"

# Initialise class



# Build the model with custom hyperparameters
ModelBuilder.build_model()

# Compile the model
ModelBuilder.compile()


ModelBuilder.summary()

# Train the model
hist = ModelBuilder.train(
    Training_data=Training_data,
    Validation_data=Validation_data,
    epochs=2,
    log_dir=logdir,
)

In [ ]:
# model = Sequential()

# l2_1 = l2(0.002)
# l2_2 = l2(0.002)
# l2_3 = l2(0.002)
# # First have an input layer, going to have 16 filters, filter is a 3x3, stride of 1
# # Relu activation turns negative values to 0, and preserves positive values
# model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (256,256,3)))
# model.add(MaxPooling2D())

# model.add(Conv2D(64, (3,3), 1, activation = 'relu'))
# model.add(MaxPooling2D())

# model.add(Conv2D(64, (3,3), 1, activation = 'relu'))
# model.add(MaxPooling2D())

# model.add(Flatten()) # condense values

# # Fully connected layers
# model.add(Dense(64, activation = 'relu', kernel_regularizer = l2_1))
# model.add(Dropout(rate =0.1))

# model.add(Dense(256, activation = 'relu', kernel_regularizer = l2_2))
# model.add(Dropout(rate=0.1))

# model.add(Dense(256, activation = 'relu', kernel_regularizer = l2_3))
# model.add(Dropout(rate=0.1))


# # Final layer that gives a single output and represets the label
# model.add(Dense(4, activation = 'softmax'))

In [ ]:
# mlflow.log_param("l2_1", l2_1)
# mlflow.log_param("l2_2", l2_2)
# mlflow.log_param("l2_3", l2_3)

## Training

In [ ]:
# logdir = '/Users/harjitgakhal/Documents/Github_Repositories/Brain-Tumour-Classification/Notebooks/logs'

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

# early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# ModelBuilder.model.fit(
#     Training_data,
#     epochs=1,
#     validation_data=Validation_data,
#     callbacks=[tensorboard_callback, early_stopping],
# )

In [ ]:
# hist = model.fit(
#     Training_data,
#     epochs=1,
#     validation_data=Validation_data,
#     callbacks=[tensorboard_callback, early_stopping],
# )

In [ ]:
hist

## Model Evaluation

In [ ]:
Plotter = Plotting_functions(class_names)

Plotter.plot_loss(hist)

In [ ]:
Plotter.plot_accuracy(hist)

## Testing data

In [ ]:
X_test, labels_testing = Cleaning.Testing_data_cleaner(Testing_data)

In [ ]:
#y_, yhat_binary, pre, re, acc = Cleaning.y_predictor(X_test,labels_testing, ModelBuilder)

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
def Testing_data_cleaner(Test_data):
    labels_testing = []
    X_test =[]

    for batch in Test_data.as_numpy_iterator():
        x_test, y_test = batch
        y_test = [np.where(row == 1)[0][0].tolist() for row in y_test] # Gets the position of the 1 to determine the type of tumor and assigns to a list
        labels_testing.extend(y_test)
        X_test.extend(x_test)

    labels_testing = np.array(labels_testing) # Full y_test labels transformed
    X_test = np.array(X_test) # Full X_test data

    return X_test, labels_testing


In [ ]:
X_test, labels_testing = Testing_data_cleaner(Testing_data)

In [ ]:
# labels_testing = []
# X_test =[]

# for batch in Testing_data.as_numpy_iterator():
#     x_test, y_test = batch
#     y_test = [np.where(row == 1)[0][0].tolist() for row in y_test] # Gets the position of the 1 to determine the type of tumor and assigns to a list
#     labels_testing.extend(y_test)
#     X_test.extend(x_test)

# labels_testing = np.array(labels_testing) # Full y_test labels transformed
# X_test = np.array(X_test) # Full X_test data

In [ ]:
def y_predictor(X_test, labels_testing):
    y_ = labels_testing # Our true y values
    yhat_ =[] # Empty list for predicted variables

    yhat = ModelBuilder.model.predict(X_test)
    yhat_binary = np.argmax(yhat, axis=1) # gets value of 1 and position
    yhat_.append(yhat_binary)

    pre.update_state(y_, yhat_binary)
    re.update_state(y_, yhat_binary)
    acc.update_state(y_, yhat_binary)

    print(f'Precision: {pre.result().numpy()}')
    print(f'Recall: {re.result().numpy()}')
    print(f'Accuracy: {acc.result().numpy()}')

    return y_, yhat_binary, pre, re, acc


In [ ]:
y_, yhat_binary, pre, re, acc = y_predictor(X_test,labels_testing)

In [ ]:
# pre = Precision()
# re = Recall()
# acc = BinaryAccuracy()

In [ ]:
# y_ = labels_testing # Our true y values
# yhat_ =[] # Empty list for predicted variables

# yhat = ModelBuilder.model.predict(X_test)
# yhat_binary = np.argmax(yhat, axis=1) # gets value of 1 and position
# yhat_.append(yhat_binary)

# pre.update_state(y_, yhat_binary)
# re.update_state(y_, yhat_binary)
# acc.update_state(y_, yhat_binary)

In [ ]:
# print(f'Precision: {pre.result().numpy()}')
# print(f'Recall: {re.result().numpy()}')
# print(f'Accuracy: {acc.result().numpy()}')

### Confusion Matrix

In [ ]:
Plotter.plot_confusion_matrix(y_, yhat_binary)

We can say that while Meningioma, Pituitary, and No tumor are being picked up well, Glioma tumor however isn't so easily and is something that would require future work

## Saving model

In [ ]:
model.save('/content/drive/MyDrive/projects/Brain_Tumor_Classification/model')